In [1]:
import numpy as np
import pandas as pd
from numpy.lib.scimath import sqrt
from scipy import pi
from scipy.stats import skewnorm
from scipy.optimize import fsolve

In [31]:
np.power(2,3.5)

11.313708498984761

In [17]:
# Strip custom rounded score file
df = pd.read_csv('./custom_rounded_score.csv')
df = df[['commentaire', 'score']]
df.to_csv('./custom_score_striped.csv', index=False)

In [2]:
# See skew distribution notebook for details
student_grade = 17.80
mean_grade = 14.28
min_grade = 8.30
max_grade = 17.8
STD = 2

In [3]:
def grade(student, mean, minimum, maximum):
  # delta
  skew = (mean - (maximum + minimum)/2) / STD

  func = lambda delta: skew - ((4 - pi)/2) * ((delta * sqrt(2/pi))**3) / (np.power(1 - (2 * delta**2) / pi, 3/2))

  delta_guess = 1
  delta_solution = fsolve(func, delta_guess)
  delta = delta_solution[0]

  # w (scale)
  w = STD / sqrt(1 - 2 * delta**2 / pi)

  # e (location)
  e = mean - w * delta * sqrt(2 / pi)

  # a (shape)
  if np.abs(delta) < 1:
    a = np.real(delta / sqrt(1 - delta**2))
  else:
    a = -np.imag(delta / sqrt(1 - delta**2))

  real_score = skewnorm.cdf(student, a, e, w)

  int_score = np.ceil(real_score * 9)  # sample real score by int from 1 to 9
  grade = (int_score - 1) / 2 + 1 # give grade between 1 and 5 (half integers)
  
  return grade

In [26]:
def comment(student, mean, minimum, maximum):
  score = grade(student, mean, minimum, maximum)

  df = pd.read_csv('./custom_score_striped.csv')
  df = df.loc[df['score'] == score, "commentaire"]

  return list(df.sample(1))

In [27]:
score = 5

In [29]:
comment(14, 12, 5, 15)

['Bon trimestre. Elève sérieuse et attentive. Continuez à consolider vos bases en compréhension de lecture.']